<a href="https://colab.research.google.com/github/weedge/doraemon-nb/blob/main/chat_bot_gpu_worker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


# install

In [11]:
!pip install "achatbot[remote_queue_chat_bot_be_worker]"="==0.0.5.27"


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 2.5 MB/s eta 0:00:00
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached apipeline-0.1.9-py3-none-any.whl.metadata (1.3 kB)
  Using cached python_dotenv-1.0.1-py3-none-any.whl.metadata (23 kB)
  Using cached pydub-0.25.1-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached pillow-10.4.0-cp310-cp310-manylinux_2_28_x86_64.whl.metadata (9.2 kB)
  Using cached aiohttp-3.10.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (7.5 kB)
  Using cached pyloudnorm-0.1.1-py3-none-any.whl.metadata (5.6 kB)
  Using cached redis-5.0.0-py3-none-any.whl.metadata (8.8 kB)
  Using cached pvporcupine-3.0.2-py3-none-any.whl.metadata (4.8 kB)
  Using cached pyannote.audio-3.2.0-py2.py3-none-any.whl.metadata (11 kB)
  Using cached webrtcvad-2.0.10.tar.gz (66 kB)
  Preparing metadata (setup.py) ... done
  Using cached torch-2.2.2-cp310-cp310-manylinux1_x86_64.whl.metadata (26 kB)
  Using cached openai-w

In [ ]:
!pip show scipy

# download models

In [4]:
!mkdir -p ~/.achatbot/{log,config,models,records,videos}

In [9]:
!git lfs install


Git LFS initialized.


In [ ]:
# vad pyannote segmentation ckpt
#!huggingface-cli download pyannote/segmentation-3.0  --local-dir ~/.achatbot/models/pyannote/segmentation-3.0 --local-dir-use-symlinks False

In [ ]:
# asr openai whisper ckpt
!wget https://openaipublic.azureedge.net/main/whisper/models/ed3a0b6b1c0edf879ad9b11b1af5a0e6ab5db9205f891f668f8b0e6c6326e34e/base.pt -O ~/.achatbot/models/base.pt

# asr hf openai whisper ckpt for transformers pipeline to load
!huggingface-cli download openai/whisper-base  --local-dir ~/.achatbot/models/openai/whisper-base --local-dir-use-symlinks False

# asr hf faster whisper (CTranslate2)
!huggingface-cli download Systran/faster-whisper-base  --local-dir ~/.achatbot/models/Systran/faster-whisper-base --local-dir-use-symlinks False

# asr SenseVoice ckpt
!huggingface-cli download FunAudioLLM/SenseVoiceSmall  --local-dir ~/.achatbot/models/FunAudioLLM/SenseVoiceSmall --local-dir-use-symlinks False

In [13]:
# llm llamacpp Qwen2-Instruct
!huggingface-cli download Qwen/Qwen2-1.5B-Instruct-GGUF qwen2-1_5b-instruct-q8_0.gguf  --local-dir ~/.achatbot/models --local-dir-use-symlinks False

# llm llamacpp Qwen1.5-chat
!huggingface-cli download Qwen/Qwen1.5-7B-Chat-GGUF qwen1_5-7b-chat-q8_0.gguf  --local-dir ~/.achatbot/models --local-dir-use-symlinks False

# llm llamacpp phi-3-mini-4k-instruct
!huggingface-cli download microsoft/Phi-3-mini-4k-instruct-gguf Phi-3-mini-4k-instruct-q4.gguf --local-dir ~/.achatbot/models --local-dir-use-symlinks False


/usr/local/lib/python3.10/dist-packages/huggingface_hub/commands/download.py:132: FutureWarning: Ignoring --local-dir-use-symlinks. Downloading to a local directory does not use symlinks anymore.
  warnings.warn(
qwen2-1_5b-instruct-q8_0.gguf: 100% 1.65G/1.65G [06:12<00:00, 4.42MB/s]
Download complete. Moving file to /root/.achatbot/models/qwen2-1_5b-instruct-q8_0.gguf
/root/.achatbot/models/qwen2-1_5b-instruct-q8_0.gguf
/usr/local/lib/python3.10/dist-packages/huggingface_hub/commands/download.py:132: FutureWarning: Ignoring --local-dir-use-symlinks. Downloading to a local directory does not use symlinks anymore.
  warnings.warn(
qwen1_5-7b-chat-q8_0.gguf: 100% 8.21G/8.21G [04:56<00:00, 27.7MB/s]
Download complete. Moving file to /root/.achatbot/models/qwen1_5-7b-chat-q8_0.gguf
/root/.achatbot/models/qwen1_5-7b-chat-q8_0.gguf
/usr/local/lib/python3.10/dist-packages/huggingface_hub/commands/download.py:132: FutureWarning: Ignoring --local-dir-use-symlinks. Downloading to a local directo

In [14]:
# tts chatTTS
!huggingface-cli download 2Noise/ChatTTS  --local-dir ~/.achatbot/models/2Noise/ChatTTS --local-dir-use-symlinks False

# tts coquiTTS
!huggingface-cli download coqui/XTTS-v2  --local-dir ~/.achatbot/models/coqui/XTTS-v2 --local-dir-use-symlinks False

# tts cosy voice
!git lfs install
!git clone https://www.modelscope.cn/iic/CosyVoice-300M.git ~/.achatbot/models/CosyVoice-300M
!git clone https://www.modelscope.cn/iic/CosyVoice-300M-SFT.git ~/.achatbot/models/CosyVoice-300M-SFT
!git clone https://www.modelscope.cn/iic/CosyVoice-300M-Instruct.git ~/.achatbot/models/CosyVoice-300M-Instruct
#git clone https://www.modelscope.cn/iic/CosyVoice-ttsfrd.git ~/.achatbot/models/CosyVoice-ttsfrd


/usr/local/lib/python3.10/dist-packages/huggingface_hub/commands/download.py:132: FutureWarning: Ignoring --local-dir-use-symlinks. Downloading to a local directory does not use symlinks anymore.
  warnings.warn(
Fetching 17 files:   0% 0/17 [00:00<?, ?it/s]Downloading 'asset/GPT.pt' to '/root/.achatbot/models/2Noise/ChatTTS/.huggingface/download/asset/GPT.pt.d7d4ee6461ea097a2be23eb40d73fb94ad3b3d39cb64fbb50cb3357fd466cadb.incomplete'

DVAE_full.pt:   0% 0.00/60.4M [00:00<?, ?B/s]

spk_stat.pt: 100% 4.26k/4.26k [00:00<00:00, 21.0MB/s]
Download complete. Moving file to /root/.achatbot/models/2Noise/ChatTTS/asset/spk_stat.pt


GPT.pt:   0% 0.00/901M [00:00<?, ?B/s]


README.md: 100% 1.93k/1.93k [00:00<00:00, 12.7MB/s]
Download complete. Moving file to /root/.achatbot/models/2Noise/ChatTTS/README.md



Vocos.pt:   0% 0.00/54.4M [00:00<?, ?B/s]



.gitattributes: 100% 1.52k/1.52k [00:00<00:00, 9.97MB/s]
Download complete. Moving file to /root/.achatbot/models/2Noise/ChatTTS/.gitattributes


# run

In [ ]:
from google.colab import userdata
redis_pwd = userdata.get('REDIS_PASSWORD')
#print(redis_pwd)

In [19]:
!ACHATBOT_PKG=1 RUN_OP=be \
  TQDM_DISABLE=True \
  REDIS_PASSWORD=$redis_pwd \
  REDIS_HOST=redis-14241.c256.us-east-1-2.ec2.redns.redis-cloud.com \
  REDIS_PORT=14241 \
  ASR_TAG=sense_voice_asr \
  ASR_LANG=zn \
  ASR_MODEL_NAME_OR_PATH=~/.achatbot/models/FunAudioLLM/SenseVoiceSmall \
  N_GPU_LAYERS=33 FLASH_ATTN=1 \
  LLM_MODEL_NAME=qwen \
  LLM_MODEL_PATH=~/.achatbot/models/qwen1_5-7b-chat-q8_0.gguf \
  TTS_TAG=tts_cosy_voice \
  python -m achatbot.cmd.remote-queue-chat.generate_audio2audio > ~/.achatbot/log/be_std_out.log


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.
/usr/local/lib/python3.10/dist-packages/torch/_jit_internal.py:741: FutureWarning: ignore(True) has been deprecated. TorchScript will now drop the function call on compilation. Use torch.jit.unused now. {}
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/diffusers/models/lora.py:393: FutureWarning: `LoRACompatibleLinear` is deprecated and will be removed in version 1.0.0. Use of `LoRACompatibleLinear` is deprecated. Please switch to PEFT backend by installing PEFT: `pip install peft`.
  deprecate("LoRACompatibleLinear", "1.0.0", deprecation_message)
/usr/local/lib/python3.10/dist-packages/torch/nn/utils/weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("to